I selected `brybnd`.

In [1]:
using LinearAlgebra
using Statistics
using OptimizationProblems, ADNLPModels, NLPModels
using DataFrames
using Optim, LineSearches

In [ ]:
function get_problem(name::String, n::Int=40)
    """
    Get a problem by name
    """
    return eval(Meta.parse("ADNLPProblems.$(name)(n=$n)"))
end

function optim_options()
    """
    Using really stict conditions in low dimensions.

    Tacking earlier in routine may be obpuscated by extra
    iterations to obtain terminal convergence.
    """
    return Optim.Options(
        iterations = 100000,
        g_abstol = eps(),     
        store_trace = false, # Trace has a lot of useful stuff...
        show_trace = false,
        extended_trace = false,
    )
end

function bfgs_alg()
    """
    Define the algorithm for the optimization
    """
    return BFGS(;
        alphaguess = LineSearches.InitialStatic(),
        linesearch = LineSearches.HagerZhang(),
        initial_invH = x -> Matrix{eltype(x)}(I, length(x), length(x)),
        manifold = Flat(),
    )
end

function qn_alg()
    """
    Defines the Quasi-Newton algorithm for the optimization.

    P is our H. Currently P = \nabla^2 f(x) = I and we fallback
    to gradient descent. 
    
    TODO: Accept P as an argument.
    """
    GradientDescent(; 
        alphaguess = LineSearches.InitialHagerZhang(),
        linesearch = LineSearches.HagerZhang(),
        P = nothing,
        precondprep = (P, x) -> nothing
    )
end

function run(problem::ADNLPModel, alg, opt)
    """
    Run the optimization algorithm on the problem
    """
    # Define objective and in-place gradient aligning with optim's interface.
    x0 = problem.meta.x0
    f(x) = obj(problem, x)
    g!(G, x) = grad!(problem, x, G) 

    # Run the optimization
    res = Optim.optimize(f, g!, x0, alg, opt)

    # Reset problem counters
    NLPModels.reset!(problem)
    
    return res
end

run (generic function with 1 method)

In [43]:
problem = get_problem("brybnd", 10)

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  SparseADHessian,
  EmptyADbackend,
}
  Problem name: brybnd
   All variables: ████████████████████ 10     All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 10                free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 10.91% sparsity)   49              linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                  

In [44]:
res = run(problem, bfgs_alg(), optim_options())

 * Status: success

 * Candidate solution
    Final objective value:     3.827224e-23

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 6.33e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.02e-17 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.28e+05 ≰ 0.0e+00
    |g(x)|                 = 3.62e-11 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    27
    f(x) calls:    76
    ∇f(x) calls:   76


In [45]:
using Arpack
H = hess(problem, res.minimizer)

λ, ϕ = eigs(H, nev=problem.meta.nvar)
println("Eigenvalues: ", λ)
println("Size of H:", size(H))

Eigenvalues: [72.40812321706545, 60.557262349013044, 58.463138515173824, 56.18002852440046, 52.13806558179139, 50.13779489745936, 44.13115956238225, 36.534796096566915, 29.38221468606604]
Size of H:(10, 10)


┌ Warning: Adjusting nev from 10 to 9
└ @ Arpack /Users/danphenderson/.julia/packages/Arpack/FCvNd/src/Arpack.jl:92


In [23]:
λ

6-element Vector{Float64}:
 82.64008325568228
 79.73419645887915
 75.31607213500006
 70.10914335584049
 65.09339845182214
 61.504873292383046

In [10]:
problem = get_problem("brybnd", 33)

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  SparseADHessian,
  EmptyADbackend,
}
  Problem name: brybnd
   All variables: ████████████████████ 33     All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 33                free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 62.57% sparsity)   210             linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                  

In [ ]:
res = run(problem, bfgs_alg(), optim_options())

 * Status: success

 * Candidate solution
    Final objective value:     1.831767e-23

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 2.40e-11 ≰ 1.0e-16
    |x - x'|/|x'|          = 3.85e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.11e-20 ≤ 1.0e-16
    |f(x) - f(x')|/|f(x')| = 3.88e+03 ≰ 0.0e+00
    |g(x)|                 = 1.59e-11 ≰ 1.0e-16

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    55
    f(x) calls:    128
    ∇f(x) calls:   128


To check that the reported minimum is a strict local minimizer we must ensure that
Df(x^*)=0 and the Hessian H=DDf[x^*] has no negative eigenvalues.